6. We continue to consider the use of a logistic regression model to predict the probability of `default` using `income` and `balance` on the `Default` data set. 
In particular, we will now compute estimates for the standard errors of the `income` and `balance` logistic regression coefficients in two different ways: (1) using the bootstrap, and (2) using the standard formula for computing the standard errors in the `glm()` function. 
Do not forget to set a random seed before beginning your analysis.

In [2]:
library(ISLR2)
library(MASS)
library(ggplot2)
library(ggthemes)
library(boot)
library(skimr)
library(caret)

# Define a color cycle to use
colors <- colorblind_pal()(8)

(a) Using the `summary()` and `glm()` functions, determine the estimated standard errors for the coefficients associated with `income` and `balance` in a multiple logistic regression model that uses both predictors.

In [3]:
head(Default)

,default,student,balance,income
,<fct>,<fct>,<dbl>,<dbl>
1,No,No,729.5265,44361.625
2,No,Yes,817.1804,12106.135
3,No,No,1073.5492,31767.139
4,No,No,529.2506,35704.494
5,No,No,785.6559,38463.496
6,No,Yes,919.5885,7491.559


In [4]:
skim(Default)

── Data Summary ────────────────────────
                           Values 
Name                       Default
Number of rows             10000  
Number of columns          4      
_______________________           
Column type frequency:            
  factor                   2      
  numeric                  2      
________________________          
Group variables            None   

── Variable type: factor ───────────────────────────────────────────────────────
  skim_variable n_missing complete_rate ordered n_unique top_counts         
1 default               0             1 FALSE          2 No: 9667, Yes: 333 
2 student               0             1 FALSE          2 No: 7056, Yes: 2944

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate   mean     sd    p0    p25    p50    p75
1 balance               0             1   835.   484.    0    482.   824.  1166.
2 income                0             1 33517. 13337

In [5]:
str(Default)

'data.frame':	10000 obs. of  4 variables:
 $ default: Factor w/ 2 levels "No","Yes": 1 1 1 1 1 1 1 1 1 1 ...
 $ student: Factor w/ 2 levels "No","Yes": 1 2 1 1 1 2 1 2 1 1 ...
 $ balance: num  730 817 1074 529 786 ...
 $ income : num  44362 12106 31767 35704 38463 ...


In [6]:
contrasts(Default$default)

,Yes
No,0
Yes,1


In [24]:
lr_fit <- glm(default ~ income + balance, data = Default, family = "binomial")
summary(lr_fit)


Call:
glm(formula = default ~ income + balance, family = "binomial", 
    data = Default)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4725  -0.1444  -0.0574  -0.0211   3.7245  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.154e+01  4.348e-01 -26.545  < 2e-16 ***
income       2.081e-05  4.985e-06   4.174 2.99e-05 ***
balance      5.647e-03  2.274e-04  24.836  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2920.6  on 9999  degrees of freedom
Residual deviance: 1579.0  on 9997  degrees of freedom
AIC: 1585

Number of Fisher Scoring iterations: 8


(b) Write a function, `boot_fn()`, that takes as input the `Default` data set as well as an `index` of the observations, and that outputs the coefficient estimates for `income` and `balance` in the multiple logistic regression model.

In [43]:
boot_fn <- function(df, index) {
    coeffs <- coef(
        glm(
            default ~ income + balance,
            data = df,
            subset = index,
            family = "binomial"
        )
    )
    return(coeffs)
}

In [44]:
# Test function
set.seed(123)
num_rows <- nrow(Default)
sampled_obs <- sample(num_rows, num_rows, replace = T)
boot_fn(Default, sampled_obs)

(Intercept)        income       balance 
-1.111670e+01  1.803436e-05  5.448714e-03

(c) Use the `boot()` function together with your `boot_fn()` function to estimate the standard errors of the logistic regression coefficients for `income` and `balance`.

We will run 1000 bootstrap with the `boot_fn` function.

In [55]:
set.seed(123)
num_bootstraps <- 1000L
boot(Default, boot_fn, num_bootstraps)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Default, statistic = boot_fn, R = num_bootstraps)


Bootstrap Statistics :
         original        bias     std. error
t1* -1.154047e+01 -2.754771e-02 4.204817e-01
t2*  2.080898e-05  1.582518e-07 4.729534e-06
t3*  5.647103e-03  1.296980e-05 2.217214e-04

The standard error estimates for $\hat{\beta}_0$, $\hat{\beta}_1$ and $\hat{\beta}_2$ obtained using the formulas using the `glm` function are 4.34e-1, 4.98e-6 and 2.27e-4, respectively. 
These are different from the estimates obtained using the bootstrap: 4.20e-1, 4.72e-6 and 2.21e-4. 
The difference can be explained recalling that the standard formulas given used in the `glm` function rely on certain assumptions:

  1. They depend on the unknown parameter $\sigma^2$, the noise variance. 
  We then estimate $\sigma^2$ using the RSS. 
  Now although the formulas for the standard errors do not rely on the linear model being correct, the estimate for $\sigma^2$ does. 
  For instance, if there is a non-linear relationship in the data, and so the residuals from a linear fit will be inflated, and so will $\sigma^2$. 

  2. The standard formulas assume (somewhat unrealistically) that the $x_i$ are fixed, and all the variability comes from the variation in the errors $\epsilon_i$.
  **The bootstrap approach does not rely on any of these assumptions, and so it is likely giving a more accurate estimate of the standard errors of $\hat{\beta}_0$, $\hat{\beta}_1$ and $\hat{\beta}_2$ than is the `summary()` function.**
